In [85]:
# Importing necessary models
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import datetime as dt
from bs4 import BeautifulSoup as bs
import requests
pd.options.display.float_format = '{:,.3f}'.format


In [119]:

# spyconst = pd.read_excel('https://us.spdrs.com/site-content/xls/SPY_All_Holdings.xls?fund=SPY&docname=All+Holdings&onyx_code1=&onyx_code2=',
#                          skiprows = 3).dropna()

# sector_list = spyconst.Sector.drop_duplicates().tolist()

# key_names = pd.DataFrame(columns = spyconst.columns.tolist())
# for sector in sector_list:
#     if sector != 'Unassigned':
#         curr_sector = spyconst[spyconst.Sector == sector].sort_values('Weight', ascending = False).head(3)
#         key_names = key_names.append(curr_sector)
        
# key_names = key_names.sort_values('Weight', ascending = False).reset_index(drop = True)

def get_stocks(filename):
    
    key_names = []
    with open(filename, mode='r', encoding='utf-8') as stock_file:
        for ticker in stock_file:
            key_names.append(ticker.strip())
            
    return key_names

def stock_summary_dfs(key_names):

    #key_names = get_stocks('interested_names.txt')
    
    
    aq_dfs = {}
    
    i = 1
    
    for ticker in key_names:#.Identifier[0:]:
        aq_url = 'https://www.alphaquery.com/stock/{}/all-data-variables'.format(ticker)
        s = requests.session()
        curr_aq = bs(s.get(aq_url).text, 'lxml')
        curr_aq = curr_aq.find('table')
        s.cookies.clear()
        key_tables = ['Profile', 'Recent Price/Volume', 'High/Low Price', 'Percent Change in Price', 
                      'Fundamental Ratios', 'Earnings', 'Dividends', 'Percent Growth in Revenue/Net Income',
                      'Percentiles Within Industry of Fundamental Ratios and Percentages',
                      'Percentiles Within Sector of Fundamental Ratios and Percentages']
    
        for row in curr_aq.find_all('tr'):
            columns = row.find_all('td')
    
            for col in columns:
                if len(columns) == 1:
                    try:
                        if len(curr_df) > 0 and curr_table in key_tables:
                            if type(aq_dfs.get(curr_table, 0)) == int:
                                aq_dfs[curr_table] = curr_df
                            else:
                                aq_dfs[curr_table] = aq_dfs[curr_table].join(curr_df)
                    except:
                        pass
                    curr_table = col.text
                    curr_df = pd.DataFrame(columns = [ticker])
                    is_field = True
                else:
                    cell_item = col.text.strip().replace(',','')
                    if is_field:
                        curr_field = cell_item
                        is_field = False
                    else:
                        try:
                            cell_item = float(cell_item)
                        except:
                            pass
    
                        try:
                            cell_time = dt.datetime.strptime(cell_item, '%Y-%m-%d')
                        except:
                            pass
    
                        curr_df.loc[curr_field] = cell_item
                        is_field = True
        
        #print("{}% Completed".format(round(i*100/len(key_names),2)))
        i += 1
        
    #print("Completed in {} seconds".format(round(time.time() - start_time, 6)))
    
    summary_dfs = {}
    
    for k,v in aq_dfs.items():
        if k == 'Profile':
            summary_dfs[k] = v.T.iloc[:,[1,3,4,5,6,8,9,10]]
            summary_dfs[k].columns = ['Name','Sector','Industry','FreeFloat Mill.','MktCap Bill.','Beta - 60 Months',
                                      'Pct Insiders Held', 'Pct Institutions Held']
            summary_dfs[k] = summary_dfs[k].sort_values('Pct Institutions Held')
            summary_dfs[k]['FreeFloat Mill.'] = pd.to_numeric(summary_dfs[k]['FreeFloat Mill.'])/10**6
            summary_dfs[k]['MktCap Bill.'] = pd.to_numeric(summary_dfs[k]['MktCap Bill.'])/10**9
            
        if k == 'Recent Price/Volume':
            summary_dfs['Prices'] = v.iloc[[0,5],:].T.apply(lambda x: pd.to_numeric(x))
        if k == 'High/Low Price':
            summary_dfs['Prices'] = summary_dfs['Prices'].join(v.T.iloc[:,[0,6,1,7,3,9]])
        if k == 'Percent Change in Price':
            v = v.T.iloc[:,[0,1,3,5]]
            v.columns = ['Return 52 Week','Return 26 Week','Return 4 Week', 'Return 1 Week']
            summary_dfs['Prices'] = summary_dfs['Prices'].join(v).sort_values('Return 1 Week', ascending = False)
        if k == 'Fundamental Ratios':
            v = v.T.iloc[:,:-2]
            v.columns = ['PE Current', 'PE TTM'] + [x.split(' (')[0] for x in v.columns.tolist()[2:]]
            summary_dfs['Fundamentals'] = v.apply(lambda x: pd.to_numeric(x))
        if k == 'Earnings':
            v = v.T.iloc[:,[1,2,5,6,8,9,10]]
            v.columns = ['Quarterly EPS Estimate','Next Earnings','Last EPS Estimate', 
                         'Last EPS',' Days Since Last Earnings','Recent FY EPS', 'Diluted EPS TTM']
            summary_dfs['Earnings'] = v
            
        if k == 'Dividends':
            v = v.tail(1).T
            v.columns = ['Div Yield']
            summary_dfs['Fundamentals'] = summary_dfs['Fundamentals'].join(v)
            
        if k == 'Percent Growth in Revenue/Net Income':
            v = v.T
            v.columns = [x.replace('Percent Growth in ', 'Change % ').replace('Quarterly','Qtr').replace('Revenue', 'Rev').replace('Income','Inc') for x in v.columns.tolist()]
            summary_dfs['Earnings'] = summary_dfs['Earnings'].join(v).sort_values('Change % Annual Rev', ascending = False)
        if k == 'Percentiles Within Industry of Fundamental Ratios and Percentages':
            v = v.T.iloc[:,[1]].apply(lambda x: pd.to_numeric(x))
            v.columns = ['TTM PE Industry Percentile']
            summary_dfs['Target Prices'] = summary_dfs['Prices'][['Closing Price']].join(summary_dfs['Fundamentals'][['PE TTM']]).join(summary_dfs['Earnings'][['Diluted EPS TTM']]).sort_values('PE TTM', ascending = False)
            summary_dfs['Target Prices'] = summary_dfs['Target Prices'].join(v)
        if k == 'Percentiles Within Sector of Fundamental Ratios and Percentages':
            v = v.T.iloc[:,[1]].apply(lambda x: pd.to_numeric(x))
            v.columns = ['TTM PE Sector Percentile']
            summary_dfs['Target Prices'] = summary_dfs['Target Prices'].join(v)
            
    summary_dfs['Target Prices']['Steal Price'] = 0.7*summary_dfs['Target Prices']['PE TTM']*summary_dfs['Target Prices']['Diluted EPS TTM']*0.9
    
    
    stock_summary = pd.concat([summary_dfs['Prices'].iloc[:,:2].T, 
                               summary_dfs['Target Prices'][['Steal Price']].T, 
                               summary_dfs['Profile'].T], axis = 0)
                
    stock_price_summary = pd.concat([summary_dfs['Prices'].T,
                                     summary_dfs['Fundamentals'].T,
                                     summary_dfs['Earnings'].T], axis = 0).iloc[2:]
                
    return stock_summary, stock_price_summary

In [121]:
key_names = ['AMZN','COST','CSCO','GOOGL','INTC','MSFT',
             'WMT','TGT','KO','NVDA']

stock_summary, stock_price_summary = stock_summary_dfs(key_names)

In [126]:
strikes = pd.concat([stock_summary.loc['Closing Price',:],
                     stock_price_summary.loc['PE Current',:]],
                    axis = 1)
strikes['PutStrike'] = np.nan
strikes.loc['AMZN','PutStrike'] = 1350
strikes.loc['COST','PutStrike'] = 275
strikes.loc['CSCO','PutStrike'] = 35
strikes.loc['GOOGL','PutStrike'] = 1000
strikes.loc['INTC','PutStrike'] = 50
strikes.loc['KO','PutStrike'] = 50
strikes.loc['MSFT','PutStrike'] = 125
strikes.loc['NVDA','PutStrike'] = 200
strikes.loc['TGT','PutStrike'] = 95
strikes.loc['WMT','PutStrike'] = 105
strikes['Drawdown'] = strikes.PutStrike/strikes['Closing Price'] - 1
strikes

,Closing Price,PE Current,PutStrike,Drawdown
AMZN,"1,975.830",69.090,"1,350.000",-0.317
COST,320.740,34.980,275.000,-0.143
CSCO,41.390,13.540,35.000,-0.154
GOOGL,"1,381.600",24.530,"1,000.000",-0.276
INTC,58.680,11.230,50.000,-0.148
KO,58.920,24.950,50.000,-0.151
MSFT,170.550,29.260,125.000,-0.267
NVDA,284.510,41.990,200.000,-0.297
TGT,107.820,15.450,95.000,-0.119
WMT,116.770,22.130,105.000,-0.101


In [118]:
# strikes = pd.concat([stock_summary.loc['Closing Price',:],
#                      stock_price_summary.loc['PE Current',:]],
#                     axis = 1)

# strikes['Current Earnings'] = strikes['Closing Price']/strikes['PE Current']
# strikes['Target PE'] = strikes['PE Current']*0.7
# strikes['Purchase Price'] = strikes['Target PE']*strikes['Current Earnings']
# strikes['Shares'] = strikes['Purchase Price'].max()/strikes['Purchase Price']
# strikes['Shares'] = strikes.Shares.apply(lambda x: np.floor(x))
# strikes

,Closing Price,PE Current,Current Earnings,Target PE,Purchase Price,Shares
AMD,46.750,50.310,0.929,35.217,32.725,40.000
AMZN,"1,908.990",70.720,26.994,49.504,"1,336.293",1.000
COST,302.730,35.720,8.475,25.004,211.911,6.000
CPB,47.880,18.940,2.528,13.258,33.516,39.000
CSCO,40.040,13.920,2.876,9.744,28.028,47.000
DIS,116.450,22.250,5.234,15.575,81.515,16.000
GOOGL,"1,337.720",25.420,52.625,17.794,936.404,1.000
INTC,55.970,11.670,4.796,8.169,39.179,34.000
KO,56.060,24.890,2.252,17.423,39.242,34.000
MSFT,164.510,30.730,5.353,21.511,115.157,11.000
